In [2]:
import pandas as pd
import numpy as np

In [8]:
kl_fileCode = pd.read_csv("kl_districtCode.csv")

In [12]:
kl_fileCode

,District,Code
0,KASARGOD,1601
1,KANNUR,1602
2,WAYANAD,1603
3,KOZHIKODE,1604
4,MALAPPURAM,1605
5,PALAKKAD,1606
6,THRISSUR,1607
7,ERNAKULAM,1608
8,IDUKKI,1609
9,KOTTAYAM,1610


In [15]:
Date = "03-05-2022" 
for idx in kl_fileCode.index:
    print(kl_fileCode["Code"][idx])
    DistrictData = pd.read_html('kerala-'+ kl_fileCode["Code"][idx] +'.html')
    DistrictData = DistrictData[1]

1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614


In [3]:
table_MN = pd.read_html('kerala-1601.html')

In [16]:
table_MN = table_MN[1]

In [20]:
table_MN[table_MN["Date"] == "03-05-2022"].columns

Index(['Date', 'Confirmed', 'Recovered', 'Active', 'Death'], dtype='object')

In [ ]:
df_districts = pd.DataFrame(columns=[])